In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK for Python: AutoML Video Classification Example

<table align="left">
<td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_AutoML_Video_Classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fsdk%2FSDK_AutoML_Video_Classification.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td> 
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/sdk/SDK_AutoML_Video_Classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td> 
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_AutoML_Video_Classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>                                                                                              
</table>

## Overview

This notebook demonstrates how to create an AutoML video classification model, with a Vertex AI video dataset, and how to serve the model for batch prediction. It requires you to provide a bucket where the dataset is stored.

Note: you may incur charges for training, prediction, storage or usage of other GCP products in connection with testing this SDK.

Learn more about [Classification for video data](https://cloud.google.com/vertex-ai/docs/training-overview#classification_for_videos).

### Objective

The objective of this notebook is to build a AutoML video classification model. The following steps have been followed:  
This tutorial uses the following Google Cloud ML services :
- Vertex AI dataset resource
- AutoML training
- Vertex AI model resource
- Vertex AI batch prediction


The steps performed include the following:

- Set your task name, and GCS prefix
- Copy AutoML video demo train data for creating managed dataset
- Create a dataset on Vertex AI.
- Configure a training job
- Launch a training job and create a model on Vertex AI
- Copy AutoML Video Demo Prediction Data for creating batch prediction job
- Perform batch prediction job on the model


### Dataset

##### HMDB: a large human motion database
Some training data and prediction data for the demo is prepared using the [HMDB Dataset](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database).

The HMDB Dataset is licensed under the Creative Commons Attribution 4.0 International License. To view a copy of this license, visit https://creativecommons.org/licenses/by/4.0/

For more information about this dataset please visit: https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/

## Costs

This tutorial uses the following billable components of Google Cloud:

- Vertex AI
- Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

LOCATION = "us-central1"  # @param {type: "string"}

#### UUID

To avoid name collisions between users on created resources, create a uuid for each session instance. Append these uuids to the respective names of the resources created in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. Vertex AI runs
the code from this package. In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. Using this model artifact, you can then
create Vertex AI Model resource and use for prediction.

In [ ]:
BUCKET_URI = (
    f"gs://your-bucket-name-{PROJECT_ID}-unique-{UUID}"  # @param {type:"string"}
)

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### Import libraries and define constants


In [ ]:
import json

from google.cloud import aiplatform, storage

MY_PROJECT = PROJECT_ID
MY_STAGING_BUCKET = BUCKET_URI

### Set Your Task Name, and GCS Prefix

If you want to centeralize all input and output files under the gcs location.

In [ ]:
TASK_TYPE = "mbsdk_automl-video-training"
PREDICTION_TYPE = "classification"
MODEL_TYPE = "CLOUD"

TASK_NAME = f"{TASK_TYPE}_{PREDICTION_TYPE}"
BUCKET_NAME = MY_STAGING_BUCKET.split("gs://")[1]
GCS_PREFIX = TASK_NAME

print(f"Bucket Name:    {BUCKET_NAME}")
print(f"Task Name:      {TASK_NAME}")

### Copy AutoML Video Demo Train Data for Creating Managed Dataset

In [ ]:
automl_video_demo_train_data = (
    "gs://automl-video-demo-data/hmdb_split1_5classes_all.csv"
)


gcs_source_train = f"gs://{BUCKET_NAME}/{TASK_NAME}/data/video_classification.csv"

!gsutil cp $automl_video_demo_train_data $gcs_source_train

## Run AutoML video training with Vertex AI video dataset

## Initialize Vertex AI SDK for Python

Initialize the *client* for Vertex AI.

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## Create a Dataset on Vertex AI Dataset resource
You now create a Vertex AI video dataset using the previously prepared csv files.

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name=f"temp-{TASK_NAME}",
    gcs_source=gcs_source_train,
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.classification,
    sync=False,
)

In [ ]:
dataset.wait()

## Launch a training job and create a model on Vertex AI

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.


### Configure a Training Job

An AutoML training pipeline is created with the `AutoMLVideoTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `prediction_type`: The type task to train the model for.
  - `classification`: A video classification model.
  - `object_tracking`: A video object tracking model.
  - `action_recognition`: A video action recognition model.

In [ ]:
job = aiplatform.AutoMLVideoTrainingJob(
    display_name=f"temp-{TASK_NAME}",
    prediction_type=PREDICTION_TYPE,
    model_type=MODEL_TYPE,
)

### Run the Training Job

Next, you run the job to start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `sync`: If set to True, the call blocks while waiting for the asynchronous batch job to complete.

The `run` method when completed returns the model resource.

The execution of the training pipeline can take over 24 hours to complete.

In [ ]:
import os
import sys

if os.getenv("IS_TESTING"):
    sys.exit(0)

In [ ]:
model = job.run(
    dataset=dataset,
    training_fraction_split=0.8,
    test_fraction_split=0.2,
    model_display_name=f"temp-{TASK_NAME}",
    sync=False,
)

In [ ]:
model.wait()

# Make a Batch Prediction request

### Copy AutoML video demo prediction data for creating batch prediction job

In [ ]:
automl_video_demo_batch_prediction_data = (
    "gs://automl-video-demo-data/hmdb_split1_predict.jsonl"
)

gcs_source_batch_prediction = (
    f"gs://{BUCKET_NAME}/{TASK_NAME}/data/video_classification_batch_prediction.jsonl"
)
gcs_destination_prefix_batch_prediction = (
    f"gs://{BUCKET_NAME}/{TASK_NAME}/batch_prediction"
)

!gsutil cp $automl_video_demo_batch_prediction_data $gcs_source_batch_prediction

### Perform batch prediction job on the model

Now that your Model resource is trained, you can make a batch prediction by invoking the batch_predict() method, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `gcs_source`: A list of one or more batch request input files.
- `gcs_destination_prefix`: The Cloud Storage location for storing the batch prediction resuls.
- `sync`: If set to True, the call blocks while waiting for the asynchronous batch job to complete.

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name=f"temp-{TASK_NAME}",
    gcs_source=gcs_source_batch_prediction,
    gcs_destination_prefix=gcs_destination_prefix_batch_prediction,
    sync=False,
)

In [ ]:
batch_predict_job.wait()

### Get the predictions

Next, get the results from the completed batch prediction job.

The results are written to the Cloud Storage output bucket you specified in the batch prediction request. You call the method iter_outputs() to get a list of each Cloud Storage file generated with the results. Each file contains one or more prediction requests in a JSON format:

- `content`: The prediction request.
- `prediction`: The prediction response.

Prediction response contains following fields

- `ids`: The internal assigned unique identifiers for each prediction request.
- `displayNames`: The class names for each class label.
- `confidences`: The predicted confidence, between 0 and 1, per class label.
- `timeSegmentStart`: The time offset in the video to the start of the video sequence.
- `timeSegmentEnd`: The time offset in the video to the end of the video sequence.

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)
client = storage.Client()
bucket = client.get_bucket(BUCKET_URI.replace("gs://", ""))
for prediction_result in prediction_results:
    gfile_name = f"{prediction_result}"
    data = bucket.blob(gfile_name).download_as_string()
    data = json.loads(data)
    print(data)

## Clean up
<a name="section-13"></a>

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

# Delete the Cloud Storage bucket
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI